In [2]:
path = r'./Data/data_practicephase_cleardev/data_practicephase_cleardev/'

## BERT

This code will fine-tune the BERT model on the training data, and then test the model on the test data, printing the accuracy of the predictions. In this example, the dataframe is expected to have two columns, one for the text (named "text") and one for the labels (named "hard_label"). The labels are expected to be in boolean format, 0 or 1.
You may need to import tokenizer from transformers library before using it.

Adopted parameters:

BERT: BertForSequenceClassification, pretrained bert-base-uncased

Epoch = 20

Optimizer: Adam Linear wormup and decay lr=2e-5, eps=1e-8

Tokenizer: bert-base-uncased


In [1]:
import pandas as pd
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer
import torch
from tqdm import tqdm
import re

### Load Data

In [4]:
def create_text_user(data):

    user_conversations = []
    
    for conversation in data:
        all_conversation = re.findall(r'"(.*?)"', conversation)
        user_conversation = all_conversation[3] + '. ' + all_conversation[7]
        user_conversations.append(user_conversation)

    return user_conversations

In [5]:
# datasets paths
convabuse_train = path + './ConvAbuse_dataset/ConvAbuse_train.json'
convabuse_dev = path + './ConvAbuse_dataset/ConvAbuse_dev.json'

brexit_path_train = path + '/HS-Brexit_dataset/HS-Brexit_train.json'
brexit_path_dev = path + '/HS-Brexit_dataset/HS-Brexit_dev.json'

MD_train = path + './MD-Agreement_dataset/MD-Agreement_train.json'
MD_dev = path + './MD-Agreement_dataset/MD-Agreement_dev.json'

# Armis dataset is not consider because the selected pre-trained bert model doesn't recognize arabic
#armis_train = path + './ArMIS_dataset/ArMIS_train.json'
#armis_dev = path + './ArMIS_dataset/ArMIS_dev.json'


In [ ]:
# import and concat train datasets
df = pd.read_json(brexit_path_train, orient='index')
df = df[['text', 'hard_label']]

df2 = pd.read_json(convabuse_train, orient='index')
df2 = df2[['text', 'hard_label']]
all = create_text_user(df2['text'])
df2['text']= all

df3 = pd.read_json(MD_train, orient='index')
df3 = df3[['text', 'hard_label']]

# concatenate df
df_tot = df.append(df2)
df_tot = df_tot.append(df3)

In [ ]:
# import and concat dev datasets
df_t = pd.read_json(brexit_path_dev, orient='index')
df_t = df_t[['text', 'hard_label']]

df2_t = pd.read_json(convabuse_dev, orient='index')
df2_t = df2_t[['text', 'hard_label']]
all = create_text_user(df2_t['text'])
df2_t['text']= all

df3_t = pd.read_json(MD_dev, orient='index')
df3_t = df3_t[['text', 'hard_label']]

#concat
df_dev = df_t.append(df2_t)
df_dev = df_dev.append(df3_t)

In [7]:
import gc
del df, df2, df3, 
del df_t, df2_t, df3_t, all
del convabuse_train, convabuse_dev ,brexit_path_train, brexit_path_dev, MD_train, MD_dev 
gc.collect()

413

In [13]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
print(torch. __version__)
#python -m pip install torch==1.7.0 -f https://download.pytorch.org/whl/torch_stable.html

1.7.0+cu110


### Define BERT Model

In [17]:
# Load the BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, output_attentions=False, output_hidden_states=False)

Downloading: 100%|██████████| 440M/440M [00:49<00:00, 8.91MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequence

In [18]:
# Move the model to the device
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Define the optimizer and schedule (linear warmup and decay)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
# Helper function for formatting inputs
def format_inputs(text, labels):
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    attention_mask = torch.tensor([[1]*len(input_ids[0])])
    labels = torch.tensor([labels])
    return input_ids, attention_mask, labels

### Fine tune the model

In [22]:
# Fine-tune the model
for epoch in tqdm(range(20)):
    model.train()
    train_loss = 0
    for i, row in df_tot.iterrows():
        input_ids, attention_mask, labels = format_inputs(row['text'], row['hard_label'])
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {train_loss/len(df_tot)}')

  5%|▌         | 1/20 [11:02<3:29:43, 662.29s/it]

Epoch 1, Loss: 0.4107941988528437


 10%|█         | 2/20 [21:41<3:16:37, 655.41s/it]

Epoch 2, Loss: 0.2960391748704165


 15%|█▌        | 3/20 [32:24<3:04:36, 651.58s/it]

...

Epoch 18, Loss: 0.41903505173759736


 95%|█████████▌| 19/20 [3:24:59<10:50, 650.28s/it]

Epoch 19, Loss: 0.4142303459766695


100%|██████████| 20/20 [3:35:43<00:00, 647.18s/it]

Epoch 20, Loss: 0.38358494355073625


### Save/load the fine-tuned model

In [28]:
torch.save(model, './model/fine_tuned_bert.pth')

In [27]:
tokenizer.save_pretrained('./tokenizer/')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json')

In [10]:
model=torch.load('./model/fine_tuned_bert.pth')

In [18]:
# if the model is on cpu
model = model.to(device)

### Test the model on Dev data

In [ ]:
# Test the model
model.eval()
test_loss = 0
for i, row in df_tot.iterrows():
    input_ids, attention_mask, labels = format_inputs(row['text'], row['hard_label'])
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs[0]
    test_loss += loss

In [12]:
# Helper function for getting predictions
def get_predictions(model, dataframe):
    predictions = []
    model.eval()
    for i, row in dataframe.iterrows():
        input_ids, attention_mask, _ = format_inputs(row['text'], row['hard_label'])
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        pred = logits.argmax(dim=1).item()
        predictions.append(pred)
    return predictions

In [19]:
# Get predictions for the test set
test_predictions = get_predictions(model, df_dev)

In [20]:
# Compare predictions to true labels
df_dev['predictions'] = test_predictions
correct_predictions = df_dev[df_dev['hard_label'] == df_dev['predictions']]
accuracy = len(correct_predictions) / len(df_dev)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7408829174664108


### Integrated Gradient

This code will use the integrated gradients method to calculate the attribution of each token in the input text to the final prediction of the model. The result is a list of values for each token, showing how much each token contributes to the final prediction.
You can also use other attribution methods such as ShapleyValue, Saliency or Deconvolution that are also available in captum library.

Integrated Gradient cod re-adapted from the Solutions proposed by vfdev-5 in https://github.com/pytorch/captum/issues/150

More details about captrum visualization at https://github.com/pytorch/captum/blob/master/captum/attr/_utils/visualization.py

Implementation details:
Number of Integrated Gradient steps = 200

In [12]:
from captum.attr import IntegratedGradients
import torch
import torch.nn as nn

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig

from captum.attr import IntegratedGradients
from captum.attr import InterpretableEmbeddingBase, TokenReferenceBase
from captum.attr import visualization
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer


# We need to split forward pass into two part: 
# 1) embeddings computation
# 2) classification

def compute_bert_outputs(model_bert, embedding_output, attention_mask=None, head_mask=None):
    if attention_mask is None:
        attention_mask = torch.ones(embedding_output.shape[0], embedding_output.shape[1]).to(embedding_output)

    extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

    extended_attention_mask = extended_attention_mask.to(dtype=next(model_bert.parameters()).dtype) # fp16 compatibility
    extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

    if head_mask is not None:
        if head_mask.dim() == 1:
            head_mask = head_mask.unsqueeze(0).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
            head_mask = head_mask.expand(model_bert.config.num_hidden_layers, -1, -1, -1, -1)
        elif head_mask.dim() == 2:
            head_mask = head_mask.unsqueeze(1).unsqueeze(-1).unsqueeze(-1)  # We can specify head_mask for each layer
        head_mask = head_mask.to(dtype=next(model_bert.parameters()).dtype) # switch to fload if need + fp16 compatibility
    else:
        head_mask = [None] * model_bert.config.num_hidden_layers

    encoder_outputs = model_bert.encoder(embedding_output,
                                         extended_attention_mask,
                                         head_mask=head_mask)
    sequence_output = encoder_outputs[0]
    pooled_output = model_bert.pooler(sequence_output)
    outputs = (sequence_output, pooled_output,) + encoder_outputs[1:]  # add hidden_states and attentions if they are here
    return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)    

In [13]:
class BertModelWrapper(nn.Module):
    
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model
        
    def forward(self, embeddings):        
        outputs = compute_bert_outputs(self.model.bert, embeddings)
        pooled_output = outputs[1]
        pooled_output = self.model.dropout(pooled_output)
        logits = self.model.classifier(pooled_output)
        return torch.softmax(logits, dim=1)[:, 1].unsqueeze(1)

In [14]:
bert_model_wrapper = BertModelWrapper(model)
ig = IntegratedGradients(bert_model_wrapper)

In [25]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

In [26]:
def interpret_sentence(model_wrapper, sentence, label=1):

    model_wrapper.eval()
    model_wrapper.zero_grad()
    
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)]).to(device)
    
    input_embedding = model_wrapper.model.bert.embeddings(input_ids)
    
    # predict
    pred = model_wrapper(input_embedding).item()
    pred_ind = round(pred)

    # compute attributions and approximation delta using integrated gradients
    attributions_ig, delta = ig.attribute(input_embedding, n_steps=200, return_convergence_delta=True)

    print('pred: ', pred_ind, '(', '%.2f' % pred, ')', ', delta: ', abs(delta))
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().data.numpy().tolist())    
    
    #add_attributions_to_visualizer(attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    return attributions_ig, tokens, pred, pred_ind, label, delta, vis_data_records_ig
    
    
def add_attributions_to_visualizer(attributions, tokens, pred, pred_ind, label, delta, vis_data_records):
    #Additional method to visualize the results achieved through interpret_sentence
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().cpu().data.numpy()
    
    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            pred_ind,
                            label,
                            "hard_label",
                            attributions.sum(),       
                            tokens[:len(attributions)],
                            delta))    


In [16]:
df_dev = df_dev.reset_index()
df_dev=df_dev[['text','hard_label']]
df_dev

,text,hard_label
0,Cheap £ means foreigners will be flocking here...,0
1,#BrexitOrNot ?? Easy!! #BREXIT and protect you...,0
2,"<user> #brexit to sum it up in just one word ""...",0
3,Putin says #Brexit reflects unhappiness with m...,0
4,#Brexit is looking likely. but anti-immigratio...,0
...,...,...
2079,Can’t let the scumbags on Wall Street try any ...,0
2080,<user> you are full of BS. #munchinLies,1
2081,"Oral sex, too. Pat, how come you didn't have a...",1
2082,<user> He’s the only President in history to d...,0


In [27]:
df_dev = df_dev.reset_index()
df_dev

,index,text,hard_label,predictions
0,1,Cheap £ means foreigners will be flocking here...,0,0
1,2,#BrexitOrNot ?? Easy!! #BREXIT and protect you...,0,0
2,3,"<user> #brexit to sum it up in just one word ""...",0,0
3,4,Putin says #Brexit reflects unhappiness with m...,0,0
4,5,#Brexit is looking likely. but anti-immigratio...,0,0
...,...,...,...,...
2079,1100,Can’t let the scumbags on Wall Street try any ...,0,0
2080,1101,<user> you are full of BS. #munchinLies,1,0
2081,1102,"Oral sex, too. Pat, how come you didn't have a...",1,0
2082,1103,<user> He’s the only President in history to d...,0,0


In [ ]:
df_dev['attention']=0
df_dev['tokens']=''
for index, row in tqdm(df_dev.iterrows()):
  attributions_ig, tokens, _, _, _, _, _ = interpret_sentence(bert_model_wrapper, sentence=df_dev.loc[index, 'text'], label=0)
  attributions = attributions_ig.sum(dim=2).squeeze(0)
  attributions = attributions / torch.norm(attributions)
  attributions = attributions.detach().cpu().data.numpy()
  df_dev.loc[index, 'attention'] = str(attributions)
  df_dev.loc[index, 'tokens'] = str(tokens)

In [ ]:
df_dev.to_csv('./Data/IG_complete_100.csv', sep='\t', index=False)

### Alternative compact code version for saving computional space

In [15]:
def interpret_sentence(model_wrapper, sentence, label=1):

    model_wrapper.eval()
    model_wrapper.zero_grad()
    
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True)]).to(device)
    
    input_embedding = model_wrapper.model.bert.embeddings(input_ids)
    
    # compute attributions and approximation delta using integrated gradients
    attributions_ig, _ = ig.attribute(input_embedding, n_steps=200, return_convergence_delta=True)
    del model_wrapper
    gc.collect()
    torch.cuda.empty_cache()

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().data.numpy().tolist())    

    return attributions_ig, tokens

In [ ]:
df_dev['attention']=0
df_dev['tokens']=''
for index, row in tqdm(df_dev.iterrows()):
  attributions_ig, tokens = interpret_sentence(bert_model_wrapper, sentence=df_dev.loc[index, 'text'], label=0)
  attributions_ig = attributions_ig.sum(dim=2).squeeze(0)
  attributions_ig = attributions_ig / torch.norm(attributions_ig)
  attributions_ig = attributions_ig.detach().cpu().data.numpy()
  df_dev.loc[index, 'attention'] = str(attributions_ig)
  df_dev.loc[index, 'tokens'] = str(tokens)

  
  del attributions_ig, tokens
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
df_dev.to_csv('./Data/IG_complete_100.csv', sep='\t', index=False)